# Исследование надежности заемщиков.


## Описание проекта

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Обзор данных

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/datasets/data.csv')
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В данных 12 столбцов. В них встречаются три разных типа данных: float64 (в двух столбцах), int64 (в пяти столбцах) и object (в пяти столбцах)

Согласно документации к данным:

* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

Столбец dept имеет тип данных int64, т.е. целые числа. Однако данные в этом столбце указывают на истинность или ложность события, 1 - имел задолженность по возврату кредита, 0 - не имел задолженности. Это логическая переменная, поэтому следует провети замену типа данных на bool. Столбцы с идентификаторами имеют уникальные значения, их не нужно сравнивать между собой, поэтому их нужно заменить на тип данных object. Также для удобства работы с данными лучше заменить вещественный тип данных в столбце total_income на целочисленный.

Также количество значений в двух столбцах отличается. Значит в них есть пропуски.

**Выводы**

В каждой строке таблицы - данные о заказчике: пол, наличие детей, семейное положение, образование, источник и количество доходов, а также цель получения кредита. Данных достаточно для проведения анализа, однако вних встречаются пропуски и несоответствующие типы данных в столбцах. Для корректного анализа следует привети данные в соответствие.

### Шаг 2.1 Заполнение пропусков

In [4]:
# узнаем количество пропусков

df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропуски в данных обнаружены в столбцах days_employed и total_income, они предоставляют информацию о трудоустройстве. Количество пропусков в обеих столбцах одинаковое, и  составляют около 10% от общего количества. Возможно они появились из-за того, что не все заявители предоставили эту информацию. Но эти данные необходимы для оценки платежеспособности заявителя, поэтому нам необходимо их заполнить.

Значение столбца total_income зависит от двух факторов:
1. Стаж работы, т.е. чем больше стаж, тем выше доход,  следовательно нужно провети категоризацию заявителей по типу занятости
2. Даже среди одной категории, у заявителей может различатся уровень дохода: у кого-то слишком низкий, у кого то слишком высокий. Поэтому среднее значение может сильно отличатся от действительности, следовательно лучше заполнить данные медианным значением

In [5]:
# посмотрим уникальные значения в столбце income_type

df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [6]:
# узнаем, есть ли пропуски в каждой категории

df_employee = df[df['income_type'] == 'сотрудник']
display(df_employee.isna().sum())
df_companion = df[df['income_type'] == 'компаньон']
display(df_companion.isna().sum())
df_retiree = df[df['income_type'] == 'пенсионер']
display(df_retiree.isna().sum())
df_civil_servant = df[df['income_type'] == 'госслужащий']
display(df_civil_servant.isna().sum())
df_unemployed = df[df['income_type'] == 'безработный']
display(df_unemployed.isna().sum())
df_entrepreneur = df[df['income_type'] == 'предприниматель']
display(df_entrepreneur.isna().sum())
df_student = df[df['income_type'] == 'студент']
display(df_student.isna().sum())
df_parental_leave = df[df['income_type'] == 'в декрете']
display(df_parental_leave.isna().sum())

children               0
days_employed       1105
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        1105
purpose                0
dtype: int64

children              0
days_employed       508
dob_years             0
education             0
education_id          0
family_status         0
family_status_id      0
gender                0
income_type           0
debt                  0
total_income        508
purpose               0
dtype: int64

children              0
days_employed       413
dob_years             0
education             0
education_id          0
family_status         0
family_status_id      0
gender                0
income_type           0
debt                  0
total_income        413
purpose               0
dtype: int64

children              0
days_employed       147
dob_years             0
education             0
education_id          0
family_status         0
family_status_id      0
gender                0
income_type           0
debt                  0
total_income        147
purpose               0
dtype: int64

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

children            0
days_employed       1
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        1
purpose             0
dtype: int64

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Пропуски есть в пяти из восьми категорий. Найдем для них медианные значения и заполним ими пропуски.

In [7]:
df['total_income'] = df.groupby('income_type')['total_income'].apply(lambda x: x.fillna(x.median()))

In [8]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

### Шаг 2.2 Проверка данных на аномалии и исправления.

In [9]:
# посмотрим уникальные значения в столбце children

df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В столбце `children` присутствуют аномальные значения: 
* 20
* -1

Они втречаются не так часто, поэтому я думаю, что эти ошибки относятся к случайным. Возможно они вызваны человеческим фактором и это опечатки, поэтому могу предположить что 20 - это 2, а -1 - это 1

In [10]:
# Заменим аномальные значения и проверим, сработал ли код
df.loc[df['children'] == 20, 'children'] = 2
df.loc[df['children'] == -1, 'children'] = 1
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [11]:
# посмотрим уникальные значения в столбце dob_years

df['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

В столбце `dob_years` тоже встречаются аномальные данные: значения 0. Таких аномалий тоже немного и возможно, эти данные не были предоставлены, и пропуски заменили на 0. Мы можем определить возраст опираясь на тип занятости, и заполнить их медианным значением. 

In [12]:
# посмотрим, в каких категориях встречается значение 0

df.loc[df['dob_years'] == 0, 'income_type'].unique()

array(['пенсионер', 'сотрудник', 'компаньон', 'госслужащий'], dtype=object)

In [13]:
# найдем медианные значения и заменим ими нули
df['dob_years'] = df.groupby('income_type')['dob_years'].apply(lambda x: x.fillna(x.median()))

df['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

При первом просмотре датасета можно заметить, что в столбце `days_employed` присутствуют отрицательные значения. Чтобы определить характер этой аномалии, нужно посмотреть, какую долю занимают эти значения.

In [14]:
# Определим количество отрицательных значений в столбце days_employed

df.loc[df['days_employed'] < 0, 'days_employed'].count()

15906

In [15]:
df.loc[df['days_employed'] < 0, 'income_type'].unique()

array(['сотрудник', 'компаньон', 'госслужащий', 'студент',
       'предприниматель', 'в декрете'], dtype=object)

Отрицательные значения фигурируют в шести из восьми категорий и занимают большую часть датасета. Можно предположить, что при выгрузке произошел сбой и данные передались со знаком минус.

Теперь посмотрим на то, как отображаются значения в двух оставшихся категориях.

In [16]:
display(df.loc[df['income_type'] == 'безработный'])
display(df.loc[df['income_type'] == 'пенсионер'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3133,1,337524.466835,31,среднее,1,женат / замужем,0,M,безработный,1,59956.991984,покупка жилья для сдачи
14798,0,395302.838654,45,Высшее,0,гражданский брак,1,F,безработный,0,202722.511368,ремонт жилью


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,118514.486412,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
...,...,...,...,...,...,...,...,...,...,...,...,...
21505,0,338904.866406,53,среднее,1,гражданский брак,1,M,пенсионер,0,75439.993167,сыграть свадьбу
21508,0,386497.714078,62,среднее,1,женат / замужем,0,M,пенсионер,0,72638.590915,недвижимость
21509,0,362161.054124,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029.059379,операции с недвижимостью
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем


Значения в столбце `days_employed` по двум другим категориям тоже сожержат аномалию. Если верить документации к данным, то значения предоставлены в днях. Однако невозможно, чтобы трудовой стаж был больше 300000 дней. Я думаю, что данные по пенсионерам и безработным были выгружены из другого источника, и представлены не в днях, а в часах.

Таким образом, для того чтобы привести данные по этому столбцу в соответствие нужно отрицательные значения изменить на положительные, а данные, записанные в часах, перевести в дни.

In [17]:
df['days_employed'] = df['days_employed'].abs()

df.loc[df['income_type'] == 'безработный', 'days_employed'] = df['days_employed'] / 24
df.loc[df['income_type'] == 'пенсионер', 'days_employed'] = df[df['income_type'] == 'пенсионер']['days_employed'] / 24

Теперь, когда данные исправлены, можно заполнить пропуски медианными значениями по категориям.

In [18]:
df['days_employed'] = df.groupby('income_type')['days_employed'].apply(lambda x: x.fillna(x.median()))
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [19]:
df['gender'].sort_values()

0          F
12867      F
12868      F
12869      F
12874      F
        ... 
3352       M
16533      M
10747      M
8572       M
10701    XNA
Name: gender, Length: 21525, dtype: object

В столбце `gender` кроме `F` и `M`, мы видим значение `XNA`. Отфильтруем строки и посмотрим как часто оно встречается.

In [20]:
df[df['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Значение `XNA` встречается всего в одной строке. Я думаю, что это просто одно пропущенное значение. Но так как оно всего одно и никак не повлияет на результат исследования, то можно оставить его как есть

### Шаг 2.3. Изменение типов данных.

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [22]:
df['days_employed'] = df['days_employed'].astype(int)
df['dob_years'] = df['dob_years'].astype(int)
df['total_income'] = df['total_income'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Шаг 2.4. Удаление дубликатов.

In [23]:
df.duplicated().sum()

54

In [24]:
df = df.drop_duplicates().reset_index(drop=True)
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21466,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21467,0,14330,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21468,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21469,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


In [25]:
df['education'].value_counts()

среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [26]:
df['education'] = df['education'].str.lower()
df['education'].value_counts()

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [27]:
df.duplicated().sum()

17

In [28]:
df = df.drop_duplicates().reset_index(drop=True)
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21450,0,14330,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21451,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21452,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


In [29]:
display(df['family_status'].value_counts())
display(df['purpose'].value_counts())

женат / замужем          12339
гражданский брак          4151
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Сначала я выбрала метод duplicated() для поиска явных дубликатов и drop_duplicates() для их удаления. После этого воспользовалась методом value_counts() для поиска неявных дубликатов. В столбце `education` были найдены одинаковые значения, записанные разным регистром. Для того, чтобы убрать эти неявные дубликаты, воспользовалась методом str.lower(). В остальных столбцах таких дубликатов обнаружено не было.

Дубликаты могли возникнуть при повторном предоставлении информации, при загрузке данных, если они были из разных источников или если пользователи вводили данные по-разному.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [30]:
education_dict = df[['education', 'education_id']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
family_status_dict = df[['family_status', 'family_status_id']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)

df = df.drop(columns=['education', 'family_status'], axis=1)
display(education_dict)
display(family_status_dict)

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


### Шаг 2.6. Категоризация дохода.

In [31]:
def income_group(income):
    if income <= 30000:
        return 'E'
    if 30001 <= income <= 50000:
        return 'D'
    if 50001 <= income <= 200000:
        return 'C'
    if 200001 <= income <= 1000000:
        return 'B'
    if income > 1000000:
        return 'A'
    
df['total_income_category'] = df['total_income'].apply(income_group)
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,14177,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


### Шаг 2.7. Категоризация целей кредита.

In [32]:
def purpose_group(purpose):
    if 'автомобил' in purpose:
        return 'операции с автомобилем'
    if 'недвижимост' in purpose or 'жиль' in purpose:
        return 'операции с недвижимостью'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'образовани' in purpose:
        return 'получение образования'
    
df['purpose_category'] = df['purpose'].apply(purpose_group)
df.head(15)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,14177,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926,27,0,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879,43,0,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152,50,1,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


### Ответы на вопросы.

##### Вопрос 1:

Есть ли зависимость между количеством детей и возвратом кредита в срок?

##### Вывод 1:

In [33]:
def union_table(df, series):
    df_pivot = df.pivot_table(index=series, columns='debt',values='days_employed', aggfunc='count')
    df_pivot['total'] = df_pivot.apply(sum, axis=1)
    df_pivot['debt_percent'] = df_pivot[1] / df_pivot['total'] * 100
    return df_pivot

union_table(df, df['children'])

debt,0,1,total,debt_percent
children,,,,
0,13028.0,1063.0,14091.0,7.543822
1,4410.0,445.0,4855.0,9.165808
2,1926.0,202.0,2128.0,9.492481
3,303.0,27.0,330.0,8.181818
4,37.0,4.0,41.0,9.756098
5,9.0,NaN,NaN,NaN


Из объединенной таблицы видно, что самый маленький процент должников приходится на людей, не имеющих детей. Однако процент людей с разным количеством детей находится примерно на одном уровне

##### Вопрос 2:

Есть ли зависимость между семейным положением и возвратом кредита в срок?

##### Вывод 2:

In [34]:
df_pivot = union_table(df, df['family_status_id'])

In [35]:
df_pivot = df_pivot.merge(family_status_dict, on='family_status_id', how='right')
df_pivot

,family_status_id,0,1,total,debt_percent,family_status
0,0,11408,931,12339,7.545182,женат / замужем
1,1,3763,388,4151,9.347145,гражданский брак
2,2,896,63,959,6.569343,вдовец / вдова
3,3,1110,85,1195,7.112971,в разводе
4,4,2536,274,2810,9.750890,Не женат / не замужем


Мы видим достаточно сильный разброс, самый маленький процент должников приходится на вдовцов, самый большой - на незамужних.

##### Вопрос 3:

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

##### Вывод 3:

In [36]:
union_table(df, df['total_income_category'])

debt,0,1,total,debt_percent
total_income_category,,,,
A,23,2,25,8.000000
B,4686,356,5042,7.060690
C,14655,1360,16015,8.492039
D,329,21,350,6.000000
E,20,2,22,9.090909


Самый высокий и самый низкий показатель приходится на категории D и E. Это люди с наиболее низкими доходами, поэтому мы не можем сказать о том, что чем выше доход, тем меньше должников.

##### Вопрос 4:

Как разные цели кредита влияют на его возврат в срок?

##### Вывод 4:

In [37]:
union_table(df, df['purpose_category'])

debt,0,1,total,debt_percent
purpose_category,,,,
операции с автомобилем,3903,403,4306,9.359034
операции с недвижимостью,10029,782,10811,7.233373
получение образования,3643,370,4013,9.220035
проведение свадьбы,2138,186,2324,8.003442


Наибольший процент должников приходится на людей, чья цель кредита - операции с автомобилем и получение образования. Наименьшую долю занимают люди бравшие кредит на операции с недвижимостью.

## Общий вывод:

Мы провели исследование на основе данных, продоставленных из банка и проверили, есть ли зависимость меджу:
* **количеством детей и возвратом кредита в срок**

Было установленно, что количество детей не влияет на возврат кредита. Доля должников находится примерно на одном уровне.

* **семейным положением и возвратом кредита в срок**

Наибольший процент должников приходится на незамужних, наименьший - на вдовцов.

* **уровнем дохода и возвратом кредита в срок**

Нет зависимости от уровня дохода, должники есть как и среди людей с высоким доходом, так и с низким.

* **целью кредита и возвратом кредита в срок**

Больше всего должников берет кредит на операции с автомобилем и получение образования, меньше всего - на операции с недвижимостью